In [168]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [169]:
#load in json
df = pd.read_json('data/elden_ring_text.json')




In [170]:
print("myack myack myack")


myack myack myack


In [171]:
#create a new df with all rows with accessories not null
columns = ['accessories', 'arts', 'gems', 'goods', 'protector', 'weapon', 'dialog']


df_accessories = df['accessories'].dropna()
df_arts = df['arts'].dropna()
df_crafting = df['gems'].dropna()
df_goods = df.dropna()
df_protector = df['protector'].dropna()
df_weapons = df['weapon'].dropna()
df_dialog = df['dialog'].dropna()


In [172]:

jp_columns = ['name_jp', 'info_jp', 'caption_jp', 'effect_jp', 'id']

def normalizeClean(df, column):
    new_df = pd.json_normalize(df[column])
    #delete and columns with 'jp' in the name
    
    new_df = new_df.drop_duplicates().drop(columns=jp_columns, errors='ignore')
    #add a columns with the name of the column
    new_df['type'] = column
    

    #add a column 
    return new_df

df_accessories = normalizeClean(df, 'accessories')
df_arts = normalizeClean(df, 'arts')
df_goods = normalizeClean(df, 'goods')
df_protector = normalizeClean(df, 'protector')
df_weapons = normalizeClean(df, 'weapon')
df_dialog = normalizeClean(df, 'dialog')
dfs = [df_accessories, df_arts, df_goods, df_protector, df_weapons, df_dialog]


In [173]:
df_goods.head()

,name_en,info_en,caption_en,effect_en,dialog_en,dialog_jp,type
0,Tarnished's Furled Finger,Creates a summon sign for cooperative multiplayer,Item for online play.\n(Can also be used from ...,NaN,NaN,NaN,goods
1,Duelist's Furled Finger,Creates a summon sign for competitive multiplayer,Item for online play.\n(Can also be used from ...,NaN,NaN,NaN,goods
2,Flask of Crimson Tears,"Restores HP, but is currently empty",A sacred flask modelled after a golden holy ch...,NaN,NaN,NaN,goods
3,Flask of Crimson Tears,Restores HP,A sacred flask modelled after a golden holy ch...,NaN,NaN,NaN,goods
4,Flask of Crimson Tears +1,"Restores HP, but is currently empty",A sacred flask modelled after a golden holy ch...,NaN,NaN,NaN,goods


In [174]:
#clean all text of any html tags or special/escape characters and numbers 
def cleanText(df):
    for column in df.columns:
        df[column] = df[column].str.replace(r'<[^>]*>', '')
        df[column] = df[column].str.replace(r'&[^;]*;', '')
        df[column] = df[column].str.replace(r'[0-9]', '')
    return df

for df in dfs:
    cleanText(df)


In [175]:
df_dialog.head(10)
type(df_dialog['info_en'][0])


str

In [176]:
import re
chunk_data = []
pattern = re.compile(r'\[(\d+)\]\s*(.*)')


for index, row in df_dialog.iterrows():
    if type(row['info_en']) is not str:
        continue
    info = row['info_en']
    name = row['name_en']
    chunks = info.split('<br/><br/>')
    count = 1
    for chunk in chunks:
        match = pattern.match(chunk.strip())
        if match:
            text = match.group(2)
            chunk_data.append({'name_en': name + str(count), 'text': text, 'type': 'dialog'})

        count += 1

print(chunk_data)

df_dialog_chunked = pd.DataFrame(chunk_data)
df_dialog_chunked.head(10)

[{'name_en': 'Melina dialog1', 'text': 'Elden Ring, O, Elden Ring.', 'type': 'dialog'}, {'name_en': 'Melina dialog2', 'text': 'Shaper of life, arbiter of fate.', 'type': 'dialog'}, {'name_en': 'Melina dialog3', 'text': 'Lord of the very stars above.', 'type': 'dialog'}, {'name_en': 'Melina dialog4', 'text': 'Tarnished warrior, spurned by the grace of gold.', 'type': 'dialog'}, {'name_en': 'Melina dialog5', 'text': 'Seek that which hath been lost to thee.', 'type': 'dialog'}, {'name_en': 'Melina dialog6', 'text': 'Across the Sea of Fog, to the Lands Between.', 'type': 'dialog'}, {'name_en': 'Melina dialog7', 'text': 'Seek the Elden Ring!', 'type': 'dialog'}, {'name_en': 'Melina dialog8', 'text': 'Have you heard of the Finger Maidens? They serve the Two Fingers, offering guidance, and aid, to the Tarnished. But you, I am afraid, are maidenless. I can play the role of maiden. Turning runes into strength. To aid you in your search for the Elden Ring. You need only take me with you. To the 

,name_en,text,type
0,Melina dialog1,"Elden Ring, O, Elden Ring.",dialog
1,Melina dialog2,"Shaper of life, arbiter of fate.",dialog
2,Melina dialog3,Lord of the very stars above.,dialog
3,Melina dialog4,"Tarnished warrior, spurned by the grace of gold.",dialog
4,Melina dialog5,Seek that which hath been lost to thee.,dialog
5,Melina dialog6,"Across the Sea of Fog, to the Lands Between.",dialog
6,Melina dialog7,Seek the Elden Ring!,dialog
7,Melina dialog8,Have you heard of the Finger Maidens? They ser...,dialog
8,Melina dialog9,Then it's settled. Summon me by grace to turn ...,dialog
9,Melina dialog10,I understand. I'm asking you to put faith in b...,dialog


In [183]:

dfs = [df_accessories, df_arts, df_goods, df_protector, df_weapons]

#join all the dataframes together on name_en
df_all = pd.concat(dfs)
#if there is coption_en and info_en, combine them

df_all['text'] = df_all['caption_en'] + ' ' + df_all['info_en']

df_all = pd.concat([df_all, df_dialog_chunked])
df_all = df_all[['name_en', 'text', 'type']]
#reset index
df_all = df_all.reset_index(drop=True)

df_all.to_pickle('data/cleaned_elden_ring_data.pkl')

In [184]:

#put df all into a json
df_all.to_json('data/cleaned_elden_ring_data.json')
